In [1]:
import os,sys
os.environ["CUDA_VISIBLE_DEVICES"]="1"
sys.path.append('..')
sys.path.append('.')
if not os.path.exists('VGDFR'):
    os.chdir('../')
# print current work dir
print(os.getcwd())
%load_ext autoreload
%autoreload 2

/mnt/public/yuanzhihang/docs/VGDFR


In [2]:

import torch
from torch.nn import functional as F
import cv2

In [3]:
from VGDFR.RIFE_model import Model

In [ ]:
model = Model()
model.load_model("data", -1)


In [5]:
model.eval()
model.device()

In [6]:
img0 = cv2.imread("/mnt/public/yuanzhihang/docs/VGDFR/Practical-RIFE/demo/I0_0.png")
img1 = cv2.imread("/mnt/public/yuanzhihang/docs/VGDFR/Practical-RIFE/demo/I0_1.png", cv2.IMREAD_UNCHANGED)

In [7]:
device=torch.device("cuda")
img0 = (torch.tensor(img0.transpose(2, 0, 1)).to(device) / 255.).unsqueeze(0) # [0,1]
img1 = (torch.tensor(img1.transpose(2, 0, 1)).to(device) / 255.).unsqueeze(0)
print(img0.min(), img0.max())

tensor(0., device='cuda:0') tensor(1., device='cuda:0')


In [8]:
n, c, h, w = img0.shape
ph = ((h - 1) // 64 + 1) * 64
pw = ((w - 1) // 64 + 1) * 64
padding = (0, pw - w, 0, ph - h)
img0 = F.pad(img0, padding)
img1 = F.pad(img1, padding)

In [9]:
img_list = [img0]        
n = 8
for i in range(n-1):
    print((i+1) * 1. / n)
    img_list.append(model.inference(img0, img1, (i+1) * 1. / n))
img_list.append(img1)


0.125
0.25
0.375
0.5
0.625
0.75
0.875


In [10]:
if not os.path.exists('output'):
    os.mkdir('output')
for i in range(len(img_list)):
    cv2.imwrite('output/img{}.png'.format(i), (img_list[i][0] * 255).byte().cpu().numpy().transpose(1, 2, 0)[:h, :w])
